In [72]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [74]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "Virny":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/Virny


# Multiple Models Interface Usage

In [113]:
import os
import pandas as pd

from virny.datasets import ACSIncomeDataset
from virny.custom_classes.metrics_interactive_visualizer import MetricsInteractiveVisualizer

In [114]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False, subsample_size=15_000, subsample_seed=42)
sensitive_attributes_dct = {'SEX': '2', 'RAC1P': ['2', '3', '4', '5', '6', '7', '8', '9'], 'SEX&RAC1P': None}

In [77]:
ROOT_DIR = os.path.join('docs', 'examples')
subgroup_metrics_df = pd.read_csv(os.path.join(ROOT_DIR, 'income_subgroup_metrics.csv'), header=0)
models_composed_metrics_df = pd.read_csv(os.path.join(ROOT_DIR, 'income_group_metrics.csv'), header=0)

In [78]:
subgroup_metrics_df['Model_Name'] = (subgroup_metrics_df['Model_Name'] + '__alpha=' +
                                     subgroup_metrics_df['Intervention_Param'].astype(str))
models_composed_metrics_df['Model_Name'] = (models_composed_metrics_df['Model_Name'] + '__alpha=' 
                                            + models_composed_metrics_df['Intervention_Param'].astype(str))

In [79]:
models_metrics_dct = dict()
for model_name in subgroup_metrics_df['Model_Name'].unique():
    models_metrics_dct[model_name] = subgroup_metrics_df[subgroup_metrics_df['Model_Name'] == model_name]

In [80]:
models_metrics_dct.keys()

dict_keys(['LGBMClassifier__alpha=0.7', 'LGBMClassifier__alpha=0.0', 'LGBMClassifier__alpha=0.4', 'LogisticRegression__alpha=0.0', 'LogisticRegression__alpha=0.7', 'LogisticRegression__alpha=0.4', 'MLPClassifier__alpha=0.0', 'MLPClassifier__alpha=0.7', 'MLPClassifier__alpha=0.4', 'RandomForestClassifier__alpha=0.4', 'RandomForestClassifier__alpha=0.7', 'RandomForestClassifier__alpha=0.0'])

## Metrics Visualization and Reporting

In [175]:
visualizer = MetricsInteractiveVisualizer(data_loader.X_data, data_loader.y_data,
                                          models_metrics_dct, models_composed_metrics_df,
                                          sensitive_attributes_dct=sensitive_attributes_dct)

In [176]:
visualizer.start_web_app()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
Keyboard interruption in main thread... closing server.


In [130]:
visualizer.stop_web_app()

Closing server running on port: 7860
